In [68]:
import pandas as pd

In [69]:
pe_file = r"C:\Users\surji\Desktop\Quant_Poject\Perfomance_metrics\data\pe_ratio_data.csv"
pb_file = r"C:\Users\surji\Desktop\Quant_Poject\Perfomance_metrics\data\pb_ratio_data.csv"

df_pe = pd.read_csv(pe_file, parse_dates=['Date'])
df_pb = pd.read_csv(pb_file, parse_dates=['Date'])

df_pe = df_pe.sort_values(['TickerName', 'Date']).reset_index(drop=True)
df_pb = df_pb.sort_values(['TickerName', 'Date']).reset_index(drop=True)

df = pd.merge(df_pe, df_pb, on=['Date', 'TickerName'], how='inner')


Rolling windows that I am testing for are [20, 60, 120, 180, 250, 500]


In [70]:

tickers = df['TickerName'].unique()
rolling_windows = [20, 60, 120, 180, 250, 500]



Changes that I made:
1. Earlier, the thresholds were strict; I made them more flexible by adding an epsilon buffer.
2. The epsilon provides flexibility in the code, especially for stocks like NVDA, whose PE and PB ratios are consistently high.
3. Even if the PE ratio is slightly above the 10th percentile, the code now still generates a signal, which earlier did not happen due to the strict threshold.
4. Thus, this adjustment helps to generate more signals and reduces missed opportunities caused by minor fluctuations around the thresholds.

In [71]:

epsilon = 5.0 

In [72]:
experiment_results=[]

for window in rolling_windows:
    print(f"Processing rolling window: {window}")
    
    df_window = df.copy()

    df_window['Rolling_P10_PE'] = df_window.groupby('TickerName')['PE'].transform(
        lambda x: x.rolling(window=window, min_periods=5).quantile(0.10)
    )
    df_window['Rolling_P90_PE'] = df_window.groupby('TickerName')['PE'].transform(
        lambda x: x.rolling(window=window, min_periods=5).quantile(0.90)
    )

    df_window['Rolling_P10_PB'] = df_window.groupby('TickerName')['PB'].transform(
        lambda x: x.rolling(window=window, min_periods=5).quantile(0.10)
    )
    df_window['Rolling_P90_PB'] = df_window.groupby('TickerName')['PB'].transform(
        lambda x: x.rolling(window=window, min_periods=5).quantile(0.90)
    )
    
    df_window['Signal'] = 0
    df_window.loc[
        (df_window['PE'] < df_window['Rolling_P10_PE']+ epsilon) |
        (df_window['PB'] < df_window['Rolling_P10_PB']+ epsilon),
        'Signal'
    ] = 1

    df_window.loc[
        (df_window['PE'] > df_window['Rolling_P90_PE']+epsilon) |
        (df_window['PB'] > df_window['Rolling_P90_PB']+epsilon),
        'Signal'
    ] = -1 

    df_window['RollingWindow'] = window
    experiment_results.append(df_window)


Processing rolling window: 20
Processing rolling window: 60
Processing rolling window: 120
Processing rolling window: 180
Processing rolling window: 250
Processing rolling window: 500


In [73]:

nvda_signals = df_window[df_window['TickerName'] == 'NVDA']['Signal'].value_counts()
tsla_signals = df_window[df_window['TickerName'] == 'TSLA']['Signal'].value_counts()



print("NVDA Signals:\n", nvda_signals)
print("Tesla Signals:\n", tsla_signals)



NVDA Signals:
 Signal
 1    22
-1    22
 0    20
Name: count, dtype: int64
Tesla Signals:
 Signal
 1    40
 0    13
-1     6
Name: count, dtype: int64


In [74]:

df_combined = pd.concat(experiment_results, ignore_index=True)

df_combined.to_csv("analysis_data/pe_pb_ratio_rolling_analysis.csv", index=False)
print("PE + PB rolling analysis file saved")

experiment_tracking = pd.DataFrame(columns=[
    'TickerName', 'RollingWindow', 'ROI', 'MaxDrawdown', 'SharpeRatio', 'ROI_to_MDD'
])
experiment_tracking.to_csv("analysis_data/experiment_tracking_pe_pb.csv", index=False)
print("File created for PE + PB experiment tracking")

PE + PB rolling analysis file saved
File created for PE + PB experiment tracking
